In [1]:
import os
import numpy as np


import pandas as pd
from matplotlib import pyplot as plt 

import ImagingRig1P as irp

%matplotlib qt
%load_ext autoreload
%autoreload 2

## Set scan information

Imaging with 450 nM LED and performing opto stimulation with 550 nm LED. Chroma ZET 488/561 filter set


In [2]:
genotype = 'test'
date = '28_10_2024'
fly = 'fly1'



In [3]:
# set stimulation parameters
scan_info = {'scan_name': 'opto_pulse',
            'led450': np.nan, # power in CoolLED units
            'led550': np.nan, # power in CoolLED units
            'sample_rate': 10000, # Hz
            'duration': 10, # s
            }

opto_stim_params = {'inter_stim_interval': 5, # seconds, time between rising edges of opto stim
                    'init_delay': 3, # seconds
                    'duration': .1, # seconds
                   }
scan_info['opto_stim_params'] = opto_stim_params

In [4]:
# number of samples in recording
n_samples = int(scan_info['sample_rate']*scan_info['duration'])

# time array
time = np.arange(0,scan_info['duration'],1./scan_info['sample_rate'])

# digital outputs
do_data = {'camera_trigger': np.zeros([n_samples,]),
           'led450_trigger': np.zeros([n_samples,]), # imaging
           'led550_trigger': np.zeros([n_samples,]), # opto
           'pump_trigger': np.zeros([n_samples,]), 
           }

# trigger camera
do_data['camera_trigger'][1:5000]=1

# imaging led
do_data['led450_trigger'][:-1]=1



# stim every 5 seconds
start_times = np.arange(opto_stim_params['init_delay'], scan_info['duration'], opto_stim_params['inter_stim_interval'])
end_times = start_times+opto_stim_params['duration']
for s, e in zip(start_times,end_times):
    t_s, t_e = np.argmin(np.abs(time-s)), np.argmin(np.abs(time-e))
    do_data['led550_trigger'][t_s:t_e]=1


# set filename
os.makedirs(f"D:\\{genotype}\\{date}\\{fly}\\", exist_ok=True)
num = 0
out_dir = f"D:\\{genotype}\\{date}\\{fly}\\{scan_info['scan_name']}_{num}.tdms"
while os.path.exists(out_dir):
    num +=1 
    out_dir = f"D:\\{genotype}\\{date}\\{fly}\\{scan_info['scan_name']}_{num}.tdms"
print(out_dir)

D:\test\28_10_2024\fly1\opto_pulse_8.tdms


### Run Experiment

# HC Image Live scan checklist


- Capture Pane:

  - Binning & SubArray 
    - Set binning to 2
    - Set sub-array size to minimal size to get desired FOV
  - Camera Control
    - Set exposure to get desired frame rate
  - Trigger Modes, Speed, & Registration
      - Make sure Speed is set to "Fast"
      - Capture Mode
        - Area
        - Internal while finding field of view
        - External: Start Trigger for data collection
      - Ouput Trigger
        - line 1
        - output trigger
        - kind=PROGRAMMABLE
        - Programmable Trigger Options: Delay=0, Period=1.0 ms, Source=READOUT END
- Sequence Pane:
  - Select Scan Type = High Speed Streaming
  - Scan Settings
    - Frame Count: Set to high enough value such that "Best Time">scan_info['duration']
    - RAM
    - check autosave
      - "..." to set save directory and filename
        - recommend to make the directory the same as tdms file
      - MPTIFF enabled

### When you're ready to collect data, set "Capture Mode" trigger to "External: Start Trigger". Hit "Start" on Sequence Pane. Then run cell below.

In [5]:
exp = irp.experiment_builder.Experiment(do_data, out_dir,
                                        sample_rate = scan_info['sample_rate'], duration = scan_info['duration'])
exp.run_tasks()

(4, 100000)


### Load TDMS file and convert to pandas dataframe

In [6]:
#convert to dataframe
df = exp.tdms_to_dataframe()


In [7]:
# save dataframe with scan info as pickle file
exp.save_dataframe(df, scan_info)

In [11]:
# check formatting of pandas array
df.head()

,time,camera_trigger,led450_trigger,led550_trigger,pump_trigger,camera_output
0,0.0000,5.014713,4.948904,0.000763,2.455170,0.008325
1,0.0001,5.006989,4.940376,0.000602,2.452757,0.009291
2,0.0002,4.999105,4.931849,0.000602,2.450183,0.008164
3,0.0003,4.991060,4.923643,0.000763,2.446643,0.008325
4,0.0004,4.983015,4.915437,0.000602,2.445517,0.008164


In [13]:
# plot 
fig, ax = plt.subplots()
ax.plot(df['time'], df['camera_trigger'])
ax.plot(df['time'], df['led450_trigger'])
ax.plot(df['time'], df['led550_trigger'])
ax.plot(df['time'], df['camera_output'])
ax.set_xlabel('Time (s)')
fig.show()

### Read imaging data that was just saved

Make sure to update mptiff_filepath to the tiff file that you saved


In [ ]:
# path to imaging file
mptiff_filepath = f"D:\\{genotype}\\{date}\\{fly}\\{scan_info['scan_name']}_.mptiff"


# load imaging data
imarr = irp.tools.open_multipage_tiff(mptiff_filepath)

cam_output_bool = df['camera_output']>1
frame_times = df['time'][np.ediff(cam_output_bool,to_end=0)>0]
frames_in_nidaq = frame_times.shape[0]

Use napari labels layer to create masks for rois you want to analyze. For example, the ring terminals in the red and green channels.

Use a unique value for each ROI you want to analyze 

In [ ]:
viewer = napari.Viewer()
viewer.add_image(imarr.mean(axis=0))
viewer.add_labels(np.zeros_like(imarr.mean(axis=0),dtype=int), name = 'rois')

After you're done creating the masks, run the cells below to extract timeseries

In [ ]:
# get masks from napari object
mask = viewer.layers['rois'].data

In [ ]:
# get number of rois
roi_vals = np.unique(mask)
roi_vals = roi_vals[roi_vals>0]

# average value within each roi
roi_arr = np.zeros((roi_vals.shape[0], imarr.shape[0]))
for row, val in enumerate(roi_vals):
    roi_arr[row, :] = np.array([imarr[t,:,:][mask==1].ravel().mean() for t in range(imarr.shape[0])])




In [ ]:
# plot
fig, ax = plt.subplots()
ax.plot(frame_times, roi_arr[:,:frames_in_nidaq].T)